In [1]:
import pandas as pd
import rdkit as rk
from PIL import Image  # Import PIL for visualization
from rdkit import Chem
from IPython.display import display
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.rdMolDescriptors import CalcNumHeteroatoms
import random #library for the generation of DNA Tags
from openpyxl import Workbook
from rdkit.Chem import SDMolSupplier, SDWriter
from rdkit.Chem import rdChemReactions
from ClickReaction import CuAAC
from rdkit.Chem.Draw import IPythonConsole  # Ensures images render properly

In [11]:
from rdkit.Chem import rdmolops

In [7]:
# Define the reactant and product SMILES
def rxn_to_smarts(rxn_file):
    # Load reaction from RXN file
    with open(rxn_file_path, "r") as file:
        rxn_block = file.read()
    
    reaction = rdChemReactions.ReactionFromRxnBlock(rxn_block)
    
    if reaction is None:
        raise ValueError("Invalid RXN file or unable to parse.")
    
    # Convert to SMARTS notation
    smarts_notation = rdChemReactions.ReactionToSmarts(reaction)
    
    return smarts_notation

# Example Usage:
rxn_file_path = "ketcher.rxn"  # Replace with your actual RXN file path
smarts = rxn_to_smarts(rxn_file_path)
print("SMARTS Notation:", smarts)

SMARTS Notation: *-[#7].*-[#6:4](-*)-[#35]>>*-[#6:4](-*)-[#7]-*.[#35]


In [7]:
# List of building blocks in SMILES format
BB = ['N=[N+]=NCC1CCCC1', 'N=[N+]=NCc1ccccc1', 'N=[N+]=NCC1CCCCCC1']

# Convert SMILES to RDKit Mol objects
molecules = []
for i, smiles in enumerate(BB):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        mol.SetProp("_Name", f"BB_{i}")  # Assign a name (important for SDF)
        AllChem.Compute2DCoords(mol)  # Compute 2D coordinates
        molecules.append(mol)
    else:
        print(f"❌ Warning: Could not parse SMILES {smiles}")

# Write valid molecules to an SDF file
sdf_filename = "azides.sdf"
with Chem.SDWriter(sdf_filename) as writer:
    for mol in molecules:
        writer.write(mol)

print(f"✅ Successfully wrote {len(molecules)} molecules to {sdf_filename}")

✅ Successfully wrote 3 molecules to azides.sdf


In [84]:
# Define the electrophile (with the leaving group Br)
electrophile_smiles = 'C#CC(Br)c1ccccc1'
electrophile = Chem.MolFromSmiles(electrophile_smiles)

# Define the nucleophile (amine group with an ether group)
nucleophile_smiles = 'Nc1ccoc1'
nucleophile = Chem.MolFromSmiles(nucleophile_smiles)

# Define the SN2 reaction SMARTS pattern
reaction_smarts = '[C:1]-Br.[N:2]>>[C:1]-[N:2]'

# Create the reaction object from SMARTS
reaction = AllChem.ReactionFromSmarts(reaction_smarts)

# Perform the reaction
reactants = [electrophile, nucleophile]
product_sets = reaction.RunReactants(reactants)

# Extract the products
products = []
for product_set in product_sets:
    for product in product_set:
        products.append(product)

In [10]:
# Define the reactants as SMILES
substrate = Chem.MolFromSmiles("C#CC(Br)c1ccccc1")  # 1-bromo-2-phenylprop-1-yne
nucleophile = Chem.MolFromSmiles("Nc1ccoc1")  # 2-aminofuran

# Define the reaction using SMARTS
reaction = AllChem.ReactionFromSmarts(
    "[C:1]([Br:2])=[C:3]#[C:4].[N:5]>>[C:1]([N:5])=[C:3]#[C:4].[Br-:2]"
)

# Run the reaction
products = reaction.RunReactants((substrate, nucleophile))

# Save the products to an SDF file
writer = Chem.SDWriter("products.sdf")
for prod_set in products:
    for prod in prod_set:
        writer.write(prod)
writer.close()

print("Products saved to products.sdf")

Products saved to products.sdf


In [6]:
# Definir los nucleófilos y electrófilos
nucleofilos = [
    Chem.MolFromSmiles("Nc1ccoc1"),  # Nucleófilo con nitrógeno
    Chem.MolFromSmiles("Oc1ccoc1"),  # Nucleófilo con oxígeno
    Chem.MolFromSmiles("Sc1cc[nH]c1"),  # Nucleófilo con azufre
    Chem.MolFromSmiles("Oc1cc[nH]c1")  # Otro nucleófilo con oxígeno
]

electrofilos = [
    Chem.MolFromSmiles("C#CC(Br)c1ccccc1"),  # Electrónfilo 1
    Chem.MolFromSmiles("C#CC(Br)C1CC1"),     # Electrónfilo 2
    Chem.MolFromSmiles("C#CC(Br)C1CCCC1")    # Electrónfilo 3
]

# Definir la reacción de sustitución nucleofílica
reaction = AllChem.ReactionFromSmarts("[C:1][Br].[N,O,S:2]>>[C:1][*:2]")

# Crear una lista para almacenar los productos
productos = []

# Iterar sobre los nucleófilos y electrófilos
for nucleofilo in nucleofilos:
    for electrofilo in electrofilos:
        # Ejecutar la reacción
        productos_reaccion = reaction.RunReactants((electrofilo, nucleofilo))
        for producto in productos_reaccion:
            productos.append(producto[0])  # Agregar el producto a la lista

# Guardar los productos en un archivo SDF
output_file = "productos.sdf"
writer = SDWriter(output_file)

for producto in productos:
    Chem.SanitizeMol(producto)  # Asegurarse de que la molécula sea válida
    writer.write(producto)

writer.close()
print(f"Se han generado {len(productos)} productos y se han guardado en el archivo {output_file}.")

Se han generado 12 productos y se han guardado en el archivo productos.sdf.


In [25]:
alkyne = Chem.MolFromSmiles("C#CC(c1ccccc1)N")
azide = Chem.MolFromSmiles("C1CCCC1C-[N-]-[N+]#N")

reaction = CuAAC(alkyne=alkyne, azide=azide)
product = reaction.get_product()

In [10]:
supp1 = Chem.SDMolSupplier('productos.sdf')

for mol in supp:
    if mol is not None: mol.GetNumAtoms()  #Create a list of molecules

alkynes = [x for x in supp]

supp2 = Chem.SDMolSupplier('azides.sdf')
for mol in supp2:
    if mol is not None: mol.GetNumAtoms()

azides = [x for x in supp2]

In [18]:
# Example lists of azides and alkynes (replace with your actual molecules)
azides = [
    Chem.MolFromSmiles("C1CCCC1C[N-]-[N+]#N"),  # Example azide
    Chem.MolFromSmiles("COC[N-]-[N+]#N")           # Another azide
]

alkynes = [
    Chem.MolFromSmiles("C#CC1=CC=CC=C1"),          # Example alkyne
    Chem.MolFromSmiles("C#CC(c1ccccc1)N") # Another alkyne
]


# List to store the products
Click_products = []

# React all azides with all alkynes
for azide in azides:
    for alkyne in alkynes:
        # Perform the CuAAC reaction
        reaction = CuAAC(alkyne, azide)
        product = reaction.get_product()
        if product:  # Ensure the reaction was successful
            Click_products.append(product)

# Print the SMILES of the products
for product in Click_products:
    print(Chem.MolToSmiles(product))

c1ccc(-c2cn(CC3CCCC3)nn2)cc1
NC(c1ccccc1)c1cn(CC2CCCC2)nn1
COCn1cc(-c2ccccc2)nn1
COCn1cc(C(N)c2ccccc2)nn1


In [36]:
# Definir los nucleófilos y electrófilos
nucleofilos = [
    Chem.MolFromSmiles("Nc1ccoc1"),  # Nucleófilo con nitrógeno
    Chem.MolFromSmiles("Oc1ccoc1"),  # Nucleófilo con oxígeno
    Chem.MolFromSmiles("Oc1cc[nH]c1")  # Otro nucleófilo con oxígeno
]

electrofilos = [
    Chem.MolFromSmiles("C#CC(Br)c1ccccc1"),  # Electrónfilo 1
    Chem.MolFromSmiles("C#CC(Br)C1CC1"),     # Electrónfilo 2
    Chem.MolFromSmiles("C#CC(Br)C1CCCC1")    # Electrónfilo 3
]

# Definir la reacción de sustitución nucleofílica
reaction = AllChem.ReactionFromSmarts("[C:1][Br].[N,O,S:2]>>[C:1][*:2]")

# Crear una lista para almacenar los productos
SN2 = []

# Iterar sobre los nucleófilos y electrófilos
for nucleofilo in nucleofilos:
    for electrofilo in electrofilos:
        # Ejecutar la reacción
        productos_reaccion = reaction.RunReactants((electrofilo, nucleofilo))
        for producto in productos_reaccion:
            SN2.append(Chem.MolToSmiles(producto[0]))  # Agregar el producto a la lista


In [37]:
SN2_products = []

for mol in SN2:
    SN2_products.append(Chem.MolFromSmiles(mol))
    
SN2_products

In [39]:
azides = [Chem.MolFromSmiles("C1CCCC1C[N-]-[N+]#N"),  # Example azide
          Chem.MolFromSmiles("COC[N-]-[N+]#N")           # Another azide
]

Click_prods = []

for alkyne in SN2_products:
    for azide in azides:
        reaction = CuAAC(alkyne, azide)
        product = reaction.get_product()
        Click_prods.append(product)

In [42]:
sdf_filename = "Combinatorial_Library.sdf"
with Chem.SDWriter(sdf_filename) as writer:
    for mol in Click_products:
        writer.write(mol)

In [43]:
len(Click_prods)

18

In [50]:
Click_prods[1], Click_prods[2]

(<rdkit.Chem.rdchem.Mol at 0x13378c970>,
 <rdkit.Chem.rdchem.Mol at 0x13378c890>)